## Antrenarea modelului CBoW

Acest notebook face parte din [Curriculum AI pentru Începători](http://aka.ms/ai-beginners)

În acest exemplu, vom analiza antrenarea unui model de limbaj CBoW pentru a obține propriul nostru spațiu de încorporare Word2Vec. Vom folosi setul de date AG News ca sursă de text.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Mai întâi să încărcăm setul nostru de date și să definim tokenizer-ul și vocabularul. Vom seta `vocab_size` la 5000 pentru a limita puțin calculele.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## Modelul CBoW

CBoW învață să prezică un cuvânt pe baza celor $2N$ cuvinte vecine. De exemplu, când $N=1$, vom obține următoarele perechi din propoziția *Îmi place să antrenez rețele*: (place,Îmi), (Îmi, place), (să, place), (place,să), (antrenez,să), (să, antrenez), (rețele, antrenez), (antrenez,rețele). Aici, primul cuvânt este cuvântul vecin folosit ca intrare, iar al doilea cuvânt este cel pe care îl prezicem.

Pentru a construi o rețea care să prezică următorul cuvânt, va trebui să furnizăm cuvântul vecin ca intrare și să obținem numărul cuvântului ca ieșire. Arhitectura rețelei CBoW este următoarea:

* Cuvântul de intrare este trecut prin stratul de încorporare. Acest strat de încorporare va fi chiar încorporarea Word2Vec, astfel încât îl vom defini separat ca variabilă `embedder`. Vom folosi o dimensiune de încorporare = 30 în acest exemplu, deși s-ar putea să doriți să experimentați cu dimensiuni mai mari (Word2Vec real are 300).
* Vectorul de încorporare va fi apoi trecut printr-un strat liniar care va prezice cuvântul de ieșire. Astfel, acesta are `vocab_size` neuroni.

Pentru ieșire, dacă folosim `CrossEntropyLoss` ca funcție de pierdere, va trebui să furnizăm doar numerele cuvintelor ca rezultate așteptate, fără codificare one-hot.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## Pregătirea datelor de antrenament

Acum să programăm funcția principală care va calcula perechile de cuvinte CBoW din text. Această funcție ne va permite să specificăm dimensiunea ferestrei și va returna un set de perechi - cuvânt de intrare și cuvânt de ieșire. Observați că această funcție poate fi utilizată atât pe cuvinte, cât și pe vectori/tensori - ceea ce ne va permite să codificăm textul înainte de a-l transmite funcției `to_cbow`.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


Să pregătim setul de date pentru antrenament. Vom parcurge toate știrile, vom apela `to_cbow` pentru a obține lista de perechi de cuvinte și vom adăuga aceste perechi în `X` și `Y`. Pentru a economisi timp, vom lua în considerare doar primele 10.000 de știri - puteți elimina cu ușurință această limitare dacă aveți mai mult timp la dispoziție și doriți să obțineți încorporări mai bune :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

Vom converti, de asemenea, acele date într-un singur set de date și vom crea un dataloader:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

Vom converti, de asemenea, acele date într-un singur set de date și vom crea un dataloader:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

Acum să trecem la antrenamentul propriu-zis. Vom folosi optimizatorul `SGD` cu o rată de învățare destul de mare. De asemenea, poți încerca să experimentezi cu alți optimizatori, cum ar fi `Adam`. Vom antrena pentru 10 epoci la început - și poți rula din nou această celulă dacă dorești o pierdere și mai mică.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Testarea Word2Vec

Pentru a utiliza Word2Vec, să extragem vectorii corespunzători tuturor cuvintelor din vocabularul nostru:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

Să vedem, de exemplu, cum cuvântul **Paris** este codificat într-un vector:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Este interesant să folosești Word2Vec pentru a căuta sinonime. Funcția următoare va returna `n` cele mai apropiate cuvinte față de un input dat. Pentru a le găsi, calculăm norma $|w_i - v|$, unde $v$ este vectorul corespunzător cuvântului nostru de intrare, iar $w_i$ este codificarea celui de-al `i`-lea cuvânt din vocabular. Apoi sortăm array-ul și returnăm indicii corespunzători folosind `argsort`, și luăm primele `n` elemente din listă, care codifică pozițiile celor mai apropiate cuvinte din vocabular.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## Concluzie

Folosind tehnici ingenioase precum CBoW, putem antrena modelul Word2Vec. De asemenea, poți încerca să antrenezi modelul skip-gram, care este conceput să prezică cuvântul vecin având în vedere cel central, și să vezi cât de bine funcționează.



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
